In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
import json
import re

service = Service(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service=service)

# ✅ Danh sách từ khoá / base_url
base_urls = [
    "https://www.cooky.vn/thanh-vien/lananhthai",
    "https://www.cooky.vn/thanh-vien/thuc_tran0028",
    "https://www.cooky.vn/thanh-vien/Diepngoc%20Nguyen",
    "https://www.cooky.vn/thanh-vien/quanmanhmultiply",
    "https://www.cooky.vn/thanh-vien/dung_tuan7062",
    "http://cooky.vn/thanh-vien/nttmyct",
    "https://www.cooky.vn/thanh-vien/hmkdesign3005",
    "https://www.cooky.vn/thanh-vien/anhquan22032006",
    "https://www.cooky.vn/thanh-vien/buitrongkhiem2010"
]
def click_load_more(driver, max_clicks=20, wait_time=2):
    for i in range(max_clicks):
        try:
            button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.recipe-more-inner a.ng-binding'))
            )
            driver.execute_script("arguments[0].click();", button)
            print(f"🟢 Đã click 'Xem thêm' lần {i+1}")
            time.sleep(wait_time)
        except (TimeoutException, NoSuchElementException):
            print("✅ Đã load hết công thức hoặc không còn nút.")
            break
        except ElementClickInterceptedException:
            print("⚠️ Nút bị che — cuộn xuống để thử lại.")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)


all_links = []

# ✅ Lặp các base_url
for base_url in base_urls:
    driver.get(base_url)
    click_load_more(driver, max_clicks=30, wait_time=2)

    links = driver.find_elements(By.CSS_SELECTOR, 'a.ng-binding')

    for link in links:
        href = link.get_attribute('href')
        if href and '/cong-thuc/' in href:
            if href.startswith('/'):
                href = f"https://www.cooky.vn{href}"
            all_links.append(href)


print(f"✅ Tổng link tìm được: {len(all_links)}")

#crawl tung link

all_recipes = []

for recipe_url in all_links:
        driver.get(recipe_url)

        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.recipe-name, h3, .recipe-desc-less'))
            )
        except Exception as e:
            print(f"⚠️ Timeout tại: {recipe_url} — Bỏ qua")
            continue
        # ID
        id_part = recipe_url.rstrip('/').split('/')[-1] if recipe_url else "Không có"

        # Tiêu đề
        try:
            raw_title = driver.find_element(By.CSS_SELECTOR, 'h3').text.strip()
            title = raw_title if raw_title else "Không có"
        except:
            title = "Không có"

        # Tác giả
        try:
            author = driver.find_element(By.CSS_SELECTOR, '.recipe-owner-name h4').text.strip()
            author = author if author else "Không có"
        except:
            author = "Không có"

        # Mô tả
        try:
            description = driver.find_element(By.CSS_SELECTOR, 'div.recipe-desc-less > p').text.strip()
            description = description if description else "Không có"
        except:
            description = "Không có"


        # Ảnh
        try:
            image_url = driver.find_element(By.CSS_SELECTOR, 'img.recipe-cover-photo').get_attribute('src')
            image_url = image_url if image_url else "Không có"
        except:
            image_url = "Không có"

        # Ingredients
        ingredients = []

        try:
            items = driver.find_elements(By.CSS_SELECTOR, '#ingredients-list .ingredient-item')
            for item in items:
                try:
                    name = item.find_element(By.CSS_SELECTOR, '.ingredient-name').text.strip()
                except:
                    name = "Không rõ"

                try:
                    quantity_text = item.find_element(By.CSS_SELECTOR, '.ingredient-quantity').text.strip()
                except:
                    quantity_text = ""

                # Xử lý tách số lượng và đơn vị
                match = re.match(r'^(\d+(?:[\.,]\d+)?)\s*(.*)$', quantity_text)
                if match:
                    quantity = match.group(1)
                    unit = match.group(2).strip() or None
                else:
                    quantity = quantity_text if quantity_text else None
                    unit = None

                ingredients.append({
                    "name": name,
                    "quantity": quantity,
                    "unit": unit,
                    "notes": None  # Nếu có thể thêm ghi chú riêng sau
                })

            if not ingredients:
                ingredients = [{"name": "Không có", "quantity": None, "unit": None, "notes": None}]
        except Exception as e:
            print("Lỗi:", e)
            ingredients = [{"name": "Không có", "quantity": None, "unit": None, "notes": None}]


        # Steps
        instructions = []

        try:
            step_items = driver.find_elements(By.CSS_SELECTOR, '.cook-step-item')
            for item in step_items:
                try:
                    desc = item.find_element(By.CSS_SELECTOR, '.step-content > p').text.strip()
                    if desc:
                        instructions.append(desc)
                except:
                    continue

            if not instructions:
                instructions = ["Không có hướng dẫn"]
        except Exception as e:
            print("Lỗi:", e)
            instructions = ["Không có hướng dẫn"]


        # Servings
        try:
            span = driver.find_element(By.XPATH, '//h3[contains(text(),"Thành phần")]/following-sibling::span')
            serving_text = span.text.strip()  # ví dụ: "Khẩu phần: 1 người"
            match = re.search(r'Khẩu phần\s*[:：]?\s*(\d+)', serving_text)
            servings = int(match.group(1)) if match else None
        except:
            servings = None

        # Time
        try:
            time_needed = driver.find_element(
                By.XPATH, '//svg[contains(@class, "mise-icon-time")]/following-sibling::span'
            ).text.strip()
            time_needed = time_needed if time_needed else "Không có"
        except:
            time_needed = "Không có"

        all_recipes.append({
            "id": id_part,
            "title": title,
            "url": recipe_url,
            "author": author,
            "description": description,
            "image": image_url,
            "ingredients": ingredients,
            "steps": instructions,
            "time": time_needed,
            "servings": servings
        })

        print(f"✅ {title}")

# ✅ Lưu file JSON
with open('recipes.json', 'w', encoding='utf-8') as f:
     json.dump(all_recipes, f, ensure_ascii=False, indent=4)

print("✅ Done!")


driver.quit()

🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
🟢 Đã click 'Xem thêm' lần 4
🟢 Đã click 'Xem thêm' lần 5
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
🟢 Đã click 'Xem thêm' lần 4
🟢 Đã click 'Xem thêm' lần 5
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
🟢 Đã click 'Xem thêm' lần 4
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
🟢 Đã click 'Xem thêm' lần 4
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
🟢 Đã click 'Xem thêm' lần 4
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
🟢 Đã click 'Xem thêm' lần 2
🟢 Đã click 'Xem thêm' lần 3
✅ Đã load hết công thức hoặc không còn nút.
🟢 Đã click 'Xem thêm' lần 1
✅ Đã loa